In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Jaison\\Documents\\Workspace\\Main Projects\\End_to_end_Employee\\End_to_End_Implementation\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Jaison\\Documents\\Workspace\\Main Projects\\End_to_end_Employee\\End_to_End_Implementation'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    train_target_path: Path
    test_target_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from Employee_Attition_End_to_end_ML_project_with_MLflow.constants import *
from Employee_Attition_End_to_end_ML_project_with_MLflow.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from Employee_Attition_End_to_end_ML_project_with_MLflow import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
       # train_data = pd.read_csv(self.config.train_data_path)
        #test_data = pd.read_csv(self.config.test_data_path)

        joblib.load(os.path.join(self.config.root_dir, "y.pkl"))
        train_x = joblib.load(os.path.join(self.config.root_dir, "X_train.pkl"))
        test_x = joblib.load(os.path.join(self.config.root_dir, "X_test.pkl"))
        train_y = joblib.load(os.path.join(self.config.root_dir, "y_train.pkl"))
        test_y = joblib.load(os.path.join(self.config.root_dir, "y_test.pkl"))


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-09-22 23:04:48,430: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-22 23:04:48,442: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-22 23:04:48,486: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-22 23:04:48,492: INFO: common: created directory at: artifacts]


[2023-09-22 23:04:48,501: INFO: common: created directory at: artifacts/model_trainer]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/model_trainer\\y.pkl'